### Импорт библиотек


In [2]:
import pandas as pd

#### Читаем данные с чанка, даем заголовки столбцам

In [12]:
df = pd.read_csv('resault/resault0', 
                names=['Transaction unique identifier',
                                 'Price', 'Date of Transfer', 'Postcode',
                                 'Property Type', 'Old/New', 'Duration',
                                 'PAON', 'SAON', 'Street', 'Locality',
                                 'Town/City', 'District','Country', 
                                 'PPD Category Type','Record Status-monthly file only'])

df.head(5)

,Unnamed: 0,Uniq Adress,Price
0,183,PRINCE OF WALES DRIVE OVERSTRAND MANSIONS LONDON,160000
1,193,ALBERT ROAD 23 ENGLEFIELD GREEN,83500
2,244,RAMSHEAD GROVE 28 LEEDS,29000
3,247,VICTORY STREET 165 PLYMOUTH,27000
4,253,WAPPING WALL TRAFALGAR COURT LONDON,195000


#### Создаем новую колонку, формируем в ней полный адрес недвижимости

In [16]:
df['Uniq Adress'] = df['Street'] + ' ' + df['PAON']
df.head()


,Transaction unique identifier,Price,Date of Transfer,Postcode,Property Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,Country,PPD Category Type,Record Status-monthly file only,Uniq Adress
0,{3DD6D1E9-68E3-4646-8682-FD7DB9884D97},76000,1995-11-30 00:00,RG21 5RX,T,N,F,1,Unnamed: 8,LANSLEY ROAD,BASINGSTOKE,BASINGSTOKE.1,BASINGSTOKE AND DEANE,HAMPSHIRE,A,A.1,LANSLEY ROAD 1
1,{82899FB9-496F-4FA3-8249-F6811A87F376},48000,1995-01-27 00:00,LA1 1SY,T,N,F,26,NaN,PORTLAND STREET,LANCASTER,LANCASTER,LANCASTER,LANCASHIRE,A,A,PORTLAND STREET 26
2,{B7B48D70-7FEB-437B-9EAC-F681547AB62F},59000,1995-11-17 00:00,PL3 6DR,S,N,F,26,NaN,FAIRVIEW AVENUE,PLYMOUTH,PLYMOUTH,PLYMOUTH,DEVON,A,A,FAIRVIEW AVENUE 26
3,{B75382B6-4B58-4284-A890-FD7E1C297E5D},69950,1995-04-06 00:00,TN2 3HU,S,N,F,227,NaN,GREGGS WOOD ROAD,TUNBRIDGE WELLS,TUNBRIDGE WELLS,TUNBRIDGE WELLS,KENT,A,A,GREGGS WOOD ROAD 227
4,{B6F968FF-CEB2-4A18-B212-EF6CFE08958D},57850,1995-12-15 00:00,HX7 5DY,S,Y,F,3,NaN,STREAMSIDE FOLD,MYTHOLMROYD,HEBDEN BRIDGE,CALDERDALE,WEST YORKSHIRE,A,A,STREAMSIDE FOLD 3


#### Группируем по адрессу и суммируем все продажи по этому адрессу 

In [17]:
df2 = df.groupby('Uniq Adress').sum() # после группировки по 'Uniq Adress'
                                      # получам сумму всех продаж по этому 
                                      # адресу 

# будем сравнивать две таблицы, новую (df2) и изначальную (df1) 
# отфильтрованную по адресу и цене продажи значит дом продавался более 1 раза

df1 = df.filter(['Uniq Adress', 'Price'], axis=1)
df2 = df2.filter(['Uniq Adress', 'Price'], axis=1)
df2.reset_index(inplace=True) # выравниваем таблицы
print('Таблица 1')
print(df1)
print('Таблица 2')
print(df2)

Таблица 1
                  Uniq Adress   Price
0              LANSLEY ROAD 1   76000
1          PORTLAND STREET 26   48000
2          FAIRVIEW AVENUE 26   59000
3        GREGGS WOOD ROAD 227   69950
4           STREAMSIDE FOLD 3   57850
...                       ...     ...
999996     MYRTLE CRESCENT 21   50000
999997       COTTRELL ROAD 40   46950
999998       BRADLEY ROAD 422  104000
999999             SKILLINS 2  147500
1000000      SCORER STREET 58   27500

[1000001 rows x 2 columns]
Таблица 2
            Uniq Adress   Price
0        11TH AVENUE 12   18950
1         23RD AVENUE 2   32000
2        24TH AVENUE 21   27000
3        24TH AVENUE 27   15000
4        26TH AVENUE 56   36500
...                 ...     ...
804782  ZORRINA CLOSE 9   95000
804783   ZOUCH STREET 8   31000
804784    ZULLA ROAD 12  145000
804785    ZULLA ROAD 24   55000
804786    ZULLA ROAD 36   80000

[804787 rows x 2 columns]


#### видно из табличек, что кол-во объектов разное

####  df1 - это изначальная таблица отфильтрованная по Полному Адресу и Цене продажи, тут адресса имеют дубликаты

#### df2- это таблица где просумированны продажи по адрессу, нет дубликатов адресса, в колонки продаж - сумма всех продаж

### Соединяем две таблицы, если цена в первой(df1) и во второй(df2) одинаковы, значит после суммирования по адрессу, не нашлось больше продаж, в этом случае, значение Price будет одинаковым со значением таблицы с проссуммированными продажами, дубликаты будут удалены, остануться уникальные значения - это и будут адресса недвижимости, которая продавалась более 1 раза, так как сумма всех продаж не равна изначальной 


In [18]:
# делаем слияние, если цена начальная и цена на сегодняшний день одинаоквая
# значит дом не продавался более 1 раза, удаляется из базы
# 

df_diff = pd.concat([df1,df2]).drop_duplicates(keep=False)
df_diff


,Uniq Adress,Price
1,PORTLAND STREET 26,48000
8,SOUTH TERRACE 2,30000
17,TWO MILE HILL ROAD 148,19250
19,BRIARY LANE 4,53000
30,CAMERON CLOSE 1,80000
...,...,...
804710,ZETLAND ROAD 48,72500
804743,ZIG ZAG ROAD 37,19000
804761,ZOAR STREET 17,28500
804764,ZOAR STREET 22,60500


In [19]:
df.to_csv('new.scv')